In [17]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio
import os
import cv2
import copy
import glob

OpenCV should be 4.8.0.76 Current version: 4.8.0


Definir parámetros de patrón

In [18]:
chessboard_shape = (7, 9)   # (num_esquinas_x, num_esquinas_y)
square_size = 20        # Opcional: tamaño real del cuadrado en cm o mm

In [19]:
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

In [20]:

imgs_path = ["../data/" + item for item in os.listdir("../data/") if item.endswith(".jpg")]
imgs = load_images(imgs_path)

In [21]:
corners = []
for img in imgs:
    ret, corner = cv2.findChessboardCorners(img, (7,9))
    corners.append((ret, corner))

In [22]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (7, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

In [23]:
imgs_copy = copy.deepcopy(imgs)

In [24]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
list_with_corners = []
for i in range(len(corners)):
    list_with_corners.append(cv2.drawChessboardCorners(imgs_copy[i], patternSize=chessboard_shape, patternWasFound=corners[i][0], corners=corners[i][1]))


In [25]:
def show_image(img, window_name="Image"):
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def write_image(img, filename):
    cv2.imwrite(filename, img)

for image in list_with_corners:
    show_image(image)


In [26]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(board_size, square_width, square_height):
    num_rows, num_cols = board_size
    chessboard_points = np.zeros((num_rows * num_cols, 3), np.float32)
    for row_idx in range(num_rows):
        for col_idx in range(num_cols):
            chessboard_points[row_idx * num_cols + col_idx, 0] = col_idx * square_width
            chessboard_points[row_idx * num_cols + col_idx, 1] = row_idx * square_height
            chessboard_points[row_idx * num_cols + col_idx, 2] = 0
    return chessboard_points

In [27]:
# TODO You need the points for every image, not just one (consider a list comprehension)
chessboard_points = get_chessboard_points((7,9), 20, 20)

In [28]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [29]:
# TODO
obj= np.array([chessboard_points]*15)
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints=obj, imagePoints=valid_corners, imageSize=imgs[0].shape[:2], cameraMatrix=np.zeros((3,3)), distCoeffs=np.zeros((1,4)))


# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [30]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)


Intrinsics:
 [[1.90576249e+04 0.00000000e+00 3.29947856e+02]
 [0.00000000e+00 2.19936370e+04 1.15324829e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[-1.54188403e+02  2.14493654e+04 -1.04148240e+00  2.90524471e-01
  -2.83945943e+04]]
Root mean squared reprojection error:
 90.49285841441701


NameError: name 'cap' is not defined